In [89]:
import pandas as pd
import numpy as np

schooldf = pd.read_csv ("schools_complete.csv")
studentdf = pd.read_csv ("students_complete.csv")

schooldf.rename_axis({'name':'school'},axis=1,inplace=True)
schoolnewdf = schooldf.copy()


In [90]:
schooldf.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [91]:
schooldf.columns.tolist()

['School ID', 'school', 'type', 'size', 'budget']

In [92]:
studentdf.head

<bound method NDFrame.head of        Student ID                 name gender grade              school  \
0               0         Paul Bradley      M   9th   Huang High School   
1               1         Victor Smith      M  12th   Huang High School   
2               2      Kevin Rodriguez      M  12th   Huang High School   
3               3    Dr. Richard Scott      M  12th   Huang High School   
4               4           Bonnie Ray      F   9th   Huang High School   
5               5        Bryan Miranda      M   9th   Huang High School   
6               6        Sheena Carter      F  11th   Huang High School   
7               7         Nicole Baker      F  12th   Huang High School   
8               8         Michael Roth      M  10th   Huang High School   
9               9       Matthew Greene      M  10th   Huang High School   
10             10     Andrew Alexander      M  10th   Huang High School   
11             11        Daniel Cooper      M  10th   Huang High Schoo

In [93]:
studentdf.columns.tolist()

['Student ID',
 'name',
 'gender',
 'grade',
 'school',
 'reading_score',
 'math_score']

In [94]:
#District Summary

Dschooldf = schooldf[schooldf['type'] == 'District']
Totalschools = Dschooldf.shape[0]

DStudenttotal = Dschooldf['size'].sum()

DBudget = Dschooldf['budget'].sum()

DSchoolStudentMerge = Dschooldf.merge(studentdf,how='outer',on='school')

DMathAVG = DSchoolStudentMerge.groupby(['type'])['math_score'].mean()[0]

DReadingAVG = DSchoolStudentMerge.groupby(['type'])['reading_score'].mean()[0]

DMathPass = DSchoolStudentMerge[DSchoolStudentMerge['math_score'] >= 70]

DReadingPass = DSchoolStudentMerge[DSchoolStudentMerge['reading_score'] >= 70]

DNumReading = DReadingPass.groupby(['type'])['Student ID'].count()[0]

DNumMath = DMathPass.groupby(['type'])['Student ID'].count()[0]

#PercentMath = (DMathPass/DStudenttotal)*100

#PercentReading = (DReadingPass/DStudenttotal)*100


DOverallPassing = (DReadingAVG+DMathAVG)/2

d = {'Total_School': [Totalschools],
     'Total_Students': [DStudenttotal],
     'Total_Budget': [DBudget],
     'Average_Math_score':[DMathAVG],
     'Average_Reading_score':[DReadingAVG],
    # '%Passing Math': [PercentMath],
    # '%Passing Reading': [PercentReading],
     '%Overall Passing Rate': [DOverallPassing]}

DSummary = pd.DataFrame(d)

DSummary = DSummary[['Total_School',
                     'Total_Students',
                     'Total_Budget',
                     'Average_Math_score',
                     'Average_Reading_score',
                     #'%Passing Math',
                    # '%Passing Reading',
                     '%Overall Passing Rate' ]]

DSummary

,Total_School,Total_Students,Total_Budget,Average_Math_score,Average_Reading_score,%Overall Passing Rate
0,7,26976,17347923,76.987026,80.962485,78.974755


In [95]:
#School Summary

Dschooldf['Per Student Budget'] = Dschooldf['budget']/Dschooldf['size']

pd.options.mode.chained_assignment = None

del Dschooldf['School ID']

AVGPassMR = studentdf.groupby(['school'])['reading_score','math_score'].mean().reset_index()

Dschooldf = Dschooldf.merge(AVGPassMR,on='school',how='outer')

PassR = studentdf[studentdf['reading_score'] >= 70]
PassM = studentdf[studentdf['math_score'] >= 70]

PassRSummary =PassR.groupby(['school'])['reading_score'].count().reset_index()
PassRSummary.rename_axis({'reading_score':'reading_count'},axis=1,inplace=True)

PassMSummary = PassM.groupby(['school'])['math_score'].count().reset_index()
PassMSummary.rename_axis({'math_score':'math_count'},axis=1,inplace=True)

PassCount = PassMSummary.merge(PassMSummary,on='school',how='inner')

Dschooldf = Dschooldf.merge(PassCount,on='school',how='outer')

#Dschooldf['% Passing Math'] = (Dschooldf['math_count']/Dschooldf['size'])*100
#Dschooldf['% Passing Reading'] = (Dschooldf['reading_count']/Dschooldf['size'])*100

#Dschooldf['% Overall Passing'] = (Dschooldf['% Passing Math'] + Dschooldf['% Passing Reading'])/2

Dschooldf.rename_axis({'reading_score':'Average Reading Score',
                       'math_score': 'Average Math Score'},axis= 1 , inplace= True)

Dschooldf
                                                    

,school,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,math_count_x,math_count_y
0,Huang High School,District,2917.0,1910635.0,655.0,81.182722,76.629414,1916,1916
1,Figueroa High School,District,2949.0,1884411.0,639.0,81.158020,76.711767,1946,1946
2,Hernandez High School,District,4635.0,3022020.0,652.0,80.934412,77.289752,3094,3094
3,Bailey High School,District,4976.0,3124928.0,628.0,81.033963,77.048432,3318,3318
4,Rodriguez High School,District,3999.0,2547363.0,637.0,80.744686,76.842711,2654,2654
5,Johnson High School,District,4761.0,3094650.0,650.0,80.966394,77.072464,3145,3145
6,Ford High School,District,2739.0,1763916.0,644.0,80.746258,77.102592,1871,1871
7,Cabrera High School,NaN,NaN,NaN,NaN,83.975780,83.061895,1749,1749
8,Griffin High School,NaN,NaN,NaN,NaN,83.816757,83.351499,1371,1371
9,Holden High School,NaN,NaN,NaN,NaN,83.814988,83.803279,395,395


In [96]:
#Top Performing Schools by passing rate
TopSchool_PassRatedf = Dschooldf.sort_values(by=['% Overall Passing'],ascending=False).head(5)

TopSchool_PassRatedf.style.background_gradient(cmap=cm,subset=['% Overall Passing'])



KeyError: '% Overall Passing'

In [ ]:
#Bottom Performing schools by passing rate

BottomSchool_PassRatedf = Dschooldf.sort_values(by=['% Overall Passing']).head(5)

BottomSchool_PassRatedf.style.background_gradient(cmap=cm,subset=['% Overall Passing'])

In [99]:
#math scores by grade
MScoreAVGdf =pd.pivot_table(studentdf,values=['math_score'],index=['school'],columns=['grade'])
MScoreAVGdf = MScoreAVGdf.reindex_axis(labels=['9th',
                                               '10th',
                                               '11th',
                                               '12th'],axis=1,level=1)
MScoreAVGdf

math_score                                 
grade                        9th       10th       11th       12th
school                                                           
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [100]:
#reading scores by grade
RScoreAVGdf=pd.pivot_table(studentdf,values=['reading_score'],index=['school'],columns=['grade'])
RScoreAVGdf = RScoreAVGdf.reindex_axis(labels=['9th',
                                               '10th',
                                               '11th',
                                               '12th'],axis=1,level=1)
RScoreAVGdf

reading_score                                 
grade                           9th       10th       11th       12th
school                                                              
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [114]:
#Scores by School Spending

SchoolSpendingdf = Dschooldf.copy()

bins= [0,585000,615000,645000,675000]
group_name=['<$585', '585 to 615', '615 to 645', '645 to 675']

pd.cut(SchoolSpendingdf["Per Student Budget"], bins, labels=group_name)
SchoolSpendingdf["Per Student Budget"] = pd.cut(SchoolSpendingdf["Per Student Budget"], bins, labels=group_name)
spending_group = SchoolSpendingdf.groupby("Per Student Budget")
spending_group.max()


,school,type,size,budget,Average Reading Score,Average Math Score,math_count_x,math_count_y
Per Student Budget,,,,,,,,
<$585,Rodriguez High School,District,4976.0,3124928.0,81.182722,77.289752,3318.0,3318.0
585 to 615,None,None,NaN,NaN,NaN,NaN,NaN,NaN
615 to 645,None,None,NaN,NaN,NaN,NaN,NaN,NaN
645 to 675,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# scores by school size

SizeScore = schooldf.copy()

label = np.array(['small','medium','large'])

SizeScore['size'] = pd.qcut(SizeScore['size'],3,labels=label)

SizeScore = SizeScore.groupby(['size'])['Average Reading Score',
                                        'Average Math Score', 
                                        #'% Passing Math',
                                        #'% Passing Reading',
                                       # '% Overall Passing'].mean().reset_index()

SizeScore = SizeScore.reindex_axis(labels=[2,1,0])
SizeScore.set_index(keys=['size'],inplace=True)

SizeScore

KeyError: "Columns not found: 'Average Math Score', '% Overall Passing', 'Average Reading Score'"

In [109]:
# scores by school type

ScoreSType = schooldf.copy()

ScoreSType = ScoreSType.groupby(['type'])['Average Reading Score',
                                          'Average Math Score',
                                          '% Passing Math',
                                          '% Passing Reading',
                                          '% Overall Passing'].mean().reset_

ScoreSType.set_index('type',inplace=True)

ScoreSType

KeyError: "Columns not found: '% Passing Math', '% Overall Passing', '% Passing Reading', 'Average Math Score', 'Average Reading Score'"